In [1]:
###predict where the link will be posted given just the title!
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

train = pd.read_csv('jan_may_2017_links_cross_posting_info.csv')
all_pages = list(train.buzz_account_display_name.unique())

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train.columns

Index(['buzz_external_id', 'buzz_post_parent_external_id',
       'buzz_post_created_at', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name', 'ext_table1_stats_date', 'ext_external_id',
       ...
       'Tasty One-Pot', 'BuzzFeed Sweaty', 'Tasty Junior',
       'Oh Great, More Politics', 'Einfach Tasty', 'BuzzFeed Steven L.',
       'The Try Guys', 'Ohmygod Yaaa', 'BuzzFeed Chloe', 'Nifty Science'],
      dtype='object', length=113)

In [3]:
starting_features = ['buzz_external_id', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name',]
df = train[starting_features]

In [9]:
###for each buzz_id, get the external ID which was posted FIRST!!!
train_grouped = train.groupby(['buzz_post_buzz_id']).agg({'buzz_post_created_at':'min'})

train_grouped = train_grouped.reset_index()

train_grouped = train_grouped.rename(columns={'buzz_post_created_at':'first_post'})

df = pd.merge(train, train_grouped, how='left', on=['buzz_post_buzz_id'])

df = df[df['buzz_post_created_at'] == df['first_post']]

In [24]:
###predict where the link will be posted given just the title!

In [50]:
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

pages = []
num_links = []
average_acc = []
average_pre = []
average_rec = []
average_f1 = []
cv_folds = 10
results = pd.DataFrame()
page_clf = []

for test_page in all_pages:
#     test_page = "BuzzFeed"
    X_1 = df[df.buzz_account_display_name == test_page]
    X_0 = df[~(df.buzz_account_display_name == test_page)].sample(n=len(X_1))
    if len(X_1) < 2.5*cv_folds:
        continue
    X_1 = X_1[['buzz_campaign_name', 'buzz_account_display_name']]
    X_0 = X_0[['buzz_campaign_name', 'buzz_account_display_name']]

    X_all = pd.concat([X_1, X_0])

    def get_class(X, page):
        if X == page:
            return 1
        else:
            return 0
    X_all['class'] = X_all['buzz_account_display_name'].apply(get_class, page = test_page)

    X_all = X_all.dropna()

    text_clf = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
                        ])

    fold = 0
    kf = KFold(n_splits = cv_folds, shuffle = True)

    accuracy = []
    precision = []
    recall = []
    fscore = []

    X = X_all['buzz_campaign_name']
    y = X_all['class']

    for train_index, test_index in kf.split(X):
#         print("Fold = ", fold)
        fold+= 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        text_clf.fit(X_train, y_train)
        y_pred = text_clf.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec, rec, fs, sup = precision_recall_fscore_support(y_test, y_pred)
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        fscore.append(fs)
    print("Page: ", test_page, "#links: ", len(X_1), "Accuracy: ", np.mean(accuracy), "Precision: ", np.mean(precision), "Recall: ", np.mean(recall), "Fscore:", np.mean(fscore))
    num_links.append(len(X_1))
    average_acc.append(np.mean(accuracy))
    average_pre.append(np.mean(precision))
    average_rec.append(np.mean(recall))
    average_f1.append(np.mean(fscore))
    pages.append(test_page)
    page_clf.append(text_clf)
results['page'] = pages
results['num_links'] = num_links
results['accuracy'] = average_acc
results['precision'] = average_pre
results['recall'] = average_rec
results['fscore'] = average_f1

Page:  BuzzFeed Quiz #links:  1316 Accuracy:  0.881848427238 Precision:  0.902321037665 Recall:  0.883009788952 Fscore: 0.880307450512
Page:  BuzzFeed Japan #links:  966 Accuracy:  0.783133913787 Precision:  0.865833765726 Recall:  0.790795198814 Fscore: 0.751409797241
Page:  BuzzFeed UK #links:  560 Accuracy:  0.683035714286 Precision:  0.767226560248 Recall:  0.68376647489 Fscore: 0.654709584089
Page:  BuzzFeed Brasil #links:  1022 Accuracy:  0.943220946915 Precision:  0.94886025067 Recall:  0.944271353994 Fscore: 0.94295810928
Page:  BuzzFeed Japan News #links:  616 Accuracy:  0.721800419617 Precision:  0.833584901167 Recall:  0.737795488652 Fscore: 0.679828366954
Page:  BuzzFeed Deutschland #links:  581 Accuracy:  0.973320070734 Precision:  0.972841210883 Recall:  0.974329853259 Fscore: 0.973129669579
Page:  BuzzFeed News #links:  649 Accuracy:  0.81824090638 Precision:  0.834572991192 Recall:  0.820735206815 Fscore: 0.81590332916
Page:  BuzzFeed #links:  978 Accuracy:  0.638532182

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Page:  BuzzFeed UK News #links:  366 Accuracy:  0.811162532395 Precision:  0.836445874121 Recall:  0.816218254474 Fscore: 0.807751165153
Page:  BuzzFeed India #links:  620 Accuracy:  0.749193548387 Precision:  0.80937220307 Recall:  0.750905144316 Fscore: 0.735298544006
Page:  BuzzFeed France News #links:  217 Accuracy:  0.926109936575 Precision:  0.934052937248 Recall:  0.930020827413 Fscore: 0.924961706114
Page:  BuzzFeed Scotland #links:  96 Accuracy:  0.708684210526 Precision:  0.775806946322 Recall:  0.708423798424 Fscore: 0.67591983064
Page:  LOLA #links:  37 Accuracy:  0.839285714286 Precision:  0.882738095238 Recall:  0.846666666667 Fscore: 0.828246753247
Page:  BuzzFeed Philippines #links:  161 Accuracy:  0.723674242424 Precision:  0.770755583788 Recall:  0.72806048634 Fscore: 0.709172874601
Page:  BuzzFeed Oz News #links:  179 Accuracy:  0.737380952381 Precision:  0.783808723067 Recall:  0.738766772331 Fscore: 0.723283771125


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Page:  BuzzFeed Reader #links:  46 Accuracy:  0.727777777778 Precision:  0.738293650794 Recall:  0.760595238095 Fscore: 0.684736929737
Page:  BuzzFeed Pink #links:  25 Accuracy:  0.84 Precision:  0.835833333333 Recall:  0.833333333333 Fscore: 0.797301587302
Page:  BuzzFeed LGBT #links:  114 Accuracy:  0.723913043478 Precision:  0.770454107081 Recall:  0.722161380286 Fscore: 0.699222646466
Page:  BuzzFeed Partner #links:  89 Accuracy:  0.68660130719 Precision:  0.704788479494 Recall:  0.693926767677 Fscore: 0.658856502832
Page:  BuzzFeed Tech #links:  211 Accuracy:  0.82430786268 Precision:  0.845696737931 Recall:  0.840815338945 Fscore: 0.821566118461
Page:  BuzzFeed Science #links:  73 Accuracy:  0.705714285714 Precision:  0.735265151515 Recall:  0.72125 Fscore: 0.694241183653


In [51]:
results

,page,num_links,accuracy,precision,recall,fscore
0,BuzzFeed Quiz,1316,0.881848,0.902321,0.883010,0.880307
1,BuzzFeed Japan,966,0.783134,0.865834,0.790795,0.751410
2,BuzzFeed UK,560,0.683036,0.767227,0.683766,0.654710
3,BuzzFeed Brasil,1022,0.943221,0.948860,0.944271,0.942958
4,BuzzFeed Japan News,616,0.721800,0.833585,0.737795,0.679828
5,BuzzFeed Deutschland,581,0.973320,0.972841,0.974330,0.973130
6,BuzzFeed News,649,0.818241,0.834573,0.820735,0.815903
7,BuzzFeed,978,0.638532,0.716346,0.640664,0.603391
8,BuzzFeed Canada,251,0.655647,0.733618,0.664959,0.626808
9,BuzzFeed Video,121,0.702667,0.779103,0.706388,0.681518


In [52]:
page_clf

[Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='c

In [147]:
#####Query by page and title of link - Should a link with this title be posted on this BF page?
# q_page = "LOLA"
q_page = "BuzzFeed Quiz"
# q_page = "BuzzFeed Japan"
q_test = ["Find Out What your spirit animal is?"]  ###list of titles you wanna query

max_prob = 0
max_page = ""

for pg in all_pages:
    try:
        clf = page_clf[pages.index(pg)]
    except:
        print("Woops, the model for page ", pg, " is still in the works!")
        continue
        
    prob = clf.predict_proba(q_test)[0][1]
    print(pg, prob)
    if prob > max_prob:
        max_prob = prob
        max_page = page

BuzzFeed Quiz 0.822006133087
BuzzFeed Japan 0.0504418176062
BuzzFeed UK 0.649453967268
BuzzFeed Brasil 0.0237510229628
BuzzFeed Japan News 0.0377321096068
BuzzFeed Deutschland 0.0372165433724
BuzzFeed News 0.415163135694
BuzzFeed 0.623760096086
BuzzFeed Canada 0.597919688812
BuzzFeed Video 0.469984381006
Cocoa Butter 0.443208450578
BuzzFeed Oz Politics 0.309079002749
Quizzes En Español 0.193981406629
Obsessed by BuzzFeed 0.480483461722
BuzzFeed News BR 0.072696859011
Buy Me That 0.38192244223
BuzzFeed UK Politics 0.376333492243
BuzzFeed Australia 0.614498944707
BuzzFeed Rewind 0.412539128523
Woops, the model for page  BuzzFeed Ladylike  is still in the works!
BuzzFeed Books 0.618130931117
BuzzFeed DIY 0.526890877113
BuzzFeed México 0.0291228070385
BuzzFeed Food 0.592997062626
BuzzFeed Celeb 0.674910157553
BuzzFeed Animals 0.693397325216
BuzzFeed France 0.0359120191369
BuzzFeed Español 0.0696082033988
BuzzFeed Style 0.63979040659
Woops, the model for page  Tasty  is still in the works!


In [145]:
max_page, max_prob

('Nifty Science', 0.82200613308712345)